In [41]:
from tools_definition import CustomShoppingSearchTool,CustomProductSearchTool
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import AzureChatOpenAI
import sys
sys.path.append('..')
from sales_training.sales_roleplaying import ScenarioRetrieval, SalesGPT,GenerateFeedback, conversation_stages
from fastapi import FastAPI, Depends, HTTPException, Request
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv()
import os

In [42]:
import re
from typing import Union

from langchain.agents import (
    AgentExecutor,
    AgentOutputParser,
    LLMSingleActionAgent,
)
from langchain.agents.agent_toolkits import NLAToolkit
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools.plugin import AIPlugin

In [43]:
os.environ['SERPAPI_API_KEY']=os.getenv('SERPAPI_API_KEY')

In [44]:
tools = [CustomShoppingSearchTool(),CustomProductSearchTool()]

In [45]:
def get_tools(query):
    tools = [CustomShoppingSearchTool(),CustomProductSearchTool()]
    return tools

In [46]:
llm = AzureChatOpenAI(
    openai_api_base=os.getenv("OPENAI_API_BASE_2"),
    openai_api_version=os.getenv('OPENAI_API_VERSION_2'),
    deployment_name=os.getenv('DEPLOYMENT_NAME_2'),
    openai_api_key=os.getenv('OPENAI_API_KEY_2'),
    openai_api_type=os.getenv('OPENAI_API_TYPE_2'),
    temperature=0
)

In [47]:
'''
product_id: 54448018888290646
title: Straight Talk Apple iPhone 11, 64gb, Black
prices: $82.68
conditions: 'New'
typical_prices: '$82.68 at Gomeboshot'
reviews: 93570
rating: 4.6
features: 
    Smartphone 
    Single SIM
    4G
    With Fast Charging
    Straight Talk
    Dual Lens
    iOS 
    1792 x 828
    Black
    Water Resistant
image_link: https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcT7Lkx4HSjKpVk_O2OBugV5tu6P_B1sWhPI8aUj_Ake7I-mUkaSsiREiKDUeC6rNsvzQigtm0SYq9xLFGeBzYv2QcIY1OMaWA&usqp=CAY
'''

"\nproduct_id: 54448018888290646\ntitle: Straight Talk Apple iPhone 11, 64gb, Black\nprices: $82.68\nconditions: 'New'\ntypical_prices: '$82.68 at Gomeboshot'\nreviews: 93570\nrating: 4.6\nfeatures: \n    Smartphone \n    Single SIM\n    4G\n    With Fast Charging\n    Straight Talk\n    Dual Lens\n    iOS \n    1792 x 828\n    Black\n    Water Resistant\nimage_link: https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcT7Lkx4HSjKpVk_O2OBugV5tu6P_B1sWhPI8aUj_Ake7I-mUkaSsiREiKDUeC6rNsvzQigtm0SYq9xLFGeBzYv2QcIY1OMaWA&usqp=CAY\n"

In [48]:
# Set up the base template
template = """Answer the following questions as best you can, you have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Output the final answer in the following format

<start format>
Product ID: <product_id>
Title: <title>
Price: <prices>
Condition: <conditions>
Typical Price: <typical_prices>
Reviews: <reviews>
Rating: <rating>
Features:
<#each features>
- <this>
</each>

Image Link: [Product Image]({{image_link}})
<end format>

Use the above format to return the putput
Question: {input}
{agent_scratchpad}"""

In [49]:
from typing import Callable

# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools_getter: Callable

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        tools = self.tools_getter(kwargs["input"])
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in tools]
        )
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

In [50]:
prompt = CustomPromptTemplate(
    template=template,
    tools_getter=get_tools,
    input_variables=["input", "intermediate_steps"],
)

In [51]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

In [52]:
output_parser = CustomOutputParser()

In [53]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names,
    return_intermediate_steps=True
)

In [57]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [58]:
op = agent_executor.run("I want to buy an Iphone")



> Entering new AgentExecutor chain...
Thought: I need to search for the product "Iphone"
Action: custom_shopping_search
Action Input: "Iphone"PARAMS HERE: {'engine': 'google_shopping', 'q': 'Iphone'}


Observation:54448018888290646
I now have the product ID for the Iphone
Action: custom_product_search
Action Input: 54448018888290646

Observation:{'product_id': 54448018888290646, 'title': 'Straight Talk Apple iPhone 11, 64gb, Black', 'prices': ['$249.00', '$249.00'], 'conditions': ['New', 'New'], 'typical_prices': {'shown_price': '$249.00 at Walmart'}, 'reviews': 93570, 'rating': 4.6, 'extensions': ['Smartphone', 'Single SIM', '4G', 'With Fast Charging', 'Straight Talk', 'Dual Lens', 'iOS', '1792 x 828', 'Black', 'Water Resistant'], 'description': 'Take it to 11. Just the right amount of everything. Colorful. Powerful. Essential. See the whole picture with Ultra Wide. iPhone 11. An essential upgrade Shoot 4K videos, beautiful portraits, and sweeping landscapes with the all-new dual-ca

In [59]:
op

'Product ID: 54448018888290646\nTitle: Straight Talk Apple iPhone 11, 64gb, Black\nPrice: $249.00\nCondition: New\nTypical Price: $249.00 at Walmart\nReviews: 93570\nRating: 4.6\nFeatures:\n- Smartphone\n- Single SIM\n- 4G\n- With Fast Charging\n- Straight Talk\n- Dual Lens\n- iOS\n- 1792 x 828\n- Black\n- Water Resistant\n\nImage Link: [Product Image](https://encrypted-tbn1.gstatic.com/shopping?q=tbn:ANd9GcT7Lkx4HSjKpVk_O2OBugV5tu6P_B1sWhPI8aUj_Ake7I-mUkaSsiREiKDUeC6rNsvzQigtm0SYq9xLFGeBzYv2QcIY1OMaWA&usqp=CAY)'

In [19]:
output_parser = CustomOutputParser()

In [6]:
agent = initialize_agent(
    [CustomShoppingSearchTool(),CustomProductSearchTool()], llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [7]:
agent.run(
    "I want to buy an iphone 14"
)



> Entering new AgentExecutor chain...
I need to search for the product "iphone 14"
Action: custom_shopping_search
Action Input: "iphone 14"PARAMS HERE: {'engine': 'google_shopping', 'q': 'iphone 14'}

Observation: 5247562739365477032
Thought:I now have the product ID for the iPhone 14
Action: custom_product_search
Action Input: 5247562739365477032
Observation: {'product_id': 5247562739365477032, 'title': 'Apple iPhone 14 - 128 GB - Purple - Unlocked', 'prices': ['$524.00', '$729.00', '$0.00 now'], 'conditions': ['New', 'New', 'New'], 'typical_prices': {'low': '$569', 'high': '$879', 'shown_price': '$524.00 at Reebelo USA'}, 'reviews': 7650, 'rating': 4.5, 'extensions': ['Smartphone', 'Dual SIM', '5G', 'With Wireless Charging', 'Unlocked', 'Dual Lens', 'iOS', 'GSM', 'Facial Recognition', 'With OLED Display'], 'description': 'With the most impressive dual-camera system on iPhone. Capture stunning photos in low light and bright light. Get peace of mind with groundbreaking safety feature

'The iPhone 14 is available for purchase with different storage options and features.'

In [18]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.utilities import GoogleSearchAPIWrapper

In [21]:
tools = [CustomShoppingSearchTool(),CustomProductSearchTool()]

In [22]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [24]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [26]:
agent_chain.run(input="Hello my name is Gautam")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: This is just an introduction, no action needed.`

In [27]:
agent_chain.run(input="What is my name")



> Entering new AgentExecutor chain...


OutputParserException: Could not parse LLM output: `Thought: I don't have access to personal information
Action: None`

In [28]:
!pip install langchain
!pip install langchain_experimental
!pip install openai
!pip install google-search-results

  Obtaining dependency information for langchain_experimental from https://files.pythonhosted.org/packages/98/18/7a30314eab8e74aa82416c6629dff8ace4226c61ff9c78ba183fcd5f29ee/langchain_experimental-0.0.31-py3-none-any.whl.metadata
  Obtaining dependency information for langchain>=0.0.308 from https://files.pythonhosted.org/packages/cb/58/27830eb8bdcfabc2ec9ce2fef4e9d4fda5ce7c886f6b5be18d2ee365e603/langchain-0.0.316-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpatch<2.0,>=1.33 from https://files.pythonhosted.org/packages/73/07/02e16ed01e04a374e644b575638ec7987ae846d25ad97bcc9945a3ee4b0e/jsonpatch-1.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.1.0,>=0.0.43 from https://files.pythonhosted.org/packages/ba/eb/4212c401170f85ff4a72f6cd8b91f9e6b0cbf17d317be40365e5bcdfbd58/langsmith-0.0.45-py3-none-any.whl.metadata
  Obtaining dependency information for jsonpointer>=1.9 from https://files.pythonhosted.org/packages/12/f6/0232cc0c617e1

In [29]:
import os
os.environ['OPENAI_API_KEY'] = str("4282343cdf384587a875631d9f8428e9")
os.environ["SERPAPI_API_KEY"] = str("96af9cb36955c3e23ba62c7776c2decb01505e3bc17247386a95db6779db485c")
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain
search = SerpAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

ModuleNotFoundError: No module named 'langchain.schema.language_model'; 'langchain.schema' is not a package